<a href="https://colab.research.google.com/github/juanigp/CT-denoising/blob/master/Demo_mask_rcnn_predictions_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MaskRCNN + template matching
## Descarga de dependencias, imports, clonar repo, abrir gdrive


In [0]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html


In [0]:
#imports
import os
import numpy as np
import cv2
from scipy import optimize,interpolate
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import drive
import torch
from utils import  make_maskrcnn_predictor, predict_image, visualize_predictions, correct_labels

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [0]:
#cloning repo
repo_url = 'https://github.com/SnoopiACK/DientesMask'
!git clone {repo_url}
repo_dir_path = os.path.basename(repo_url)
%cd {repo_dir_path}

Cloning into 'DientesMask'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 723 (delta 51), reused 91 (delta 13), pack-reused 557
Receiving objects: 100% (723/723), 311.12 MiB | 36.52 MiB/s, done.
Resolving deltas: 100% (239/239), done.
Checking out files: 100% (456/456), done.
/content/DientesMask/DientesMask


In [0]:
#mount drive
drive.mount('/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
#set MetadataCatalog (used in visualization)
teeth_classes=["M","P","C","I"]
MetadataCatalog.get("teeth_types").set(thing_classes=teeth_classes)
dataset_metadata_1 = MetadataCatalog.get("teeth_types")

teeth_numbers = ["18","17","16","15","14","13","12","11","21","22","23","24","25","26","27","28",\
                 "48","47","46","45","44","43","42","41","31","32","33","34","35","36","37","38"]
MetadataCatalog.get("teeth_numbers").set(thing_classes=teeth_numbers)
dataset_metadata_2 = MetadataCatalog.get("teeth_numbers")

## Cargar modelo y predecir clases

In [0]:
img_path = "data/images/neyra_karen_02012019_112241.jpg"
model_path = r"/gdrive/My Drive/models/mask_rcnn/model_final.pth"
num_classes = 4
th = 0.5

#cargar modelo
predictor = make_maskrcnn_predictor(model_path, num_classes, th)
#predecir imagen
predictions = predict_image(img_path)
#viz
visualize_img_and_predictions(img, predictions, dataset_metadata_1)

## Corregir el etiquetado

In [0]:
predictions = correct_labels(predictions)
visualize_img_and_predictions(img, predictions, dataset_metadata_2)